# Setup

In [ ]:
# set up snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Introduction to Complete

In [ ]:
from snowflake.cortex import Complete

Complete("llama3.1-405b", "how do snowflakes get their unique patterns?")

# Introduction to Task-specific Functions

In [ ]:
from snowflake.cortex import Translate

Translate("Snowflakes get their unique patterns through a combination of temperature, humidity, and air currents in the atmosphere.", "en", "fr")

In [ ]:
from snowflake.cortex import Translate

Translate("Les flocons de neige obtiennent leurs modèles uniques grâce à une combinaison de température, d’humidité et de courants atmosphériques.", "", "en")

# Create data for batch analysis

In [ ]:
-- Create the table
CREATE OR REPLACE TABLE call_transcripts (
    CallID INT, 
    Transcript VARCHAR
);

-- Insert values into the table
INSERT INTO call_transcripts VALUES
    (1, 'The billing system is fantastic! It''s so clear and easy to use.'),
    (2, 'Your billing department is a disaster! I''ve been overcharged three times this month.'),
    (3, 'Adding a new line was so smooth! Your team made it a breeze.'),
    (4, 'I''ve never had such a horrible time trying to add a new line. The process is broken.'),
    (5, 'The roaming fee explanation was perfect! I appreciate the clarity.'),
    (6, 'These roaming fees are outrageous! I can''t believe how much I was charged.'),
    (7, 'Billing support was amazing! They resolved my issue within minutes.'),
    (8, 'Adding a new line has been an absolute nightmare. I regret ever trying.'),
    (9, 'I''m impressed by how helpful the roaming fee customer service was. They answered everything.'),
    (10, 'I''ve never been this disappointed with roaming charges. It''s completely unfair.');

# Sentiment

In [ ]:
CREATE OR REPLACE TABLE call_transcripts AS
SELECT *, SNOWFLAKE.CORTEX.SENTIMENT(transcript) AS SENTIMENT
FROM call_transcripts;

SELECT * FROM call_transcripts LIMIT 10;

# Summarize

In [ ]:
from snowflake.cortex import Summarize

text = """
A snowflake is a single ice crystal that has achieved a sufficient size, and may have amalgamated with others, which falls through the Earth's atmosphere as snow.
Each flake nucleates around a tiny particle in supersaturated air masses by attracting supercooled cloud water droplets, which freeze and accrete in crystal form. 
Snow appears white in color despite being made of clear ice. 
This is due to diffuse reflection of the whole spectrum of light by the small crystal facets of the snowflakes.
"""

Summarize(text)


# Text Classification

In [ ]:
SELECT *,
SNOWFLAKE.CORTEX.CLASSIFY_TEXT(transcript, ['Billing','Add new line', 'Roaming fees']) AS text_classify_output
FROM call_transcripts;

In [ ]:
SELECT *,
SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
  transcript,
  [
    {
      'label': 'Billing',
      'description': 'questions or issues related to billing',
      'examples': [
        'Hi, I noticed an extra charge on my account this month. Could you please clarify what it’s for?',
        'I need help updating my payment method. My current card is no longer valid.',
        'Can you explain why my subscription fee increased this month? I wasn’t notified of any changes.'
      ]
    },
    {
      'label': 'Add new line',
      'description': 'questions or issues related to adding a new phone line or service',
      'examples': [
        'I’d like to add a new line to my account. Can you help with the process?',
        'What’s the cost of adding a new line to my current plan?',
        'I need to add a line for my family member. Can you assist with the setup?'
      ]
    },
    {
      'label': 'Roaming fees',
      'description': 'questions or issues related to roaming charges on a phone plan',
      'examples': [
        'Why was I charged for roaming while traveling abroad?',
        'Can you explain how international roaming works on my plan?',
        'I noticed unexpected charges for roaming. Can you clarify these?'
      ]
    }
  ],
  {'task_description': 'Return a classification of the type of issue described in the request'}
) AS text_classify_output
FROM call_transcripts;


In [ ]:
from snowflake.cortex import ClassifyText

ClassifyText("How can I buy a ticket for the Subway?", ["how to", "recommendations"])

In [ ]:
ClassifyText("What is the best broadway show to see right now?", ["how to", "recommendations"])

# Using Complete

In [ ]:
from snowflake.cortex import Complete

Complete("llama3.1-405b", "¿Cómo adquieren los copos de nieve su forma única?")

# Options: Cortex Guard

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant that answers programming questions in the style of a rancher from the western United States.' },
    {'role': 'user', 'content': 'Are semicolons optional in JavaScript?'}
]

options = {
    'guardrails': True,
}

Complete("llama3.1-405b", messages, options = options)

# Multi-turn chat

In [ ]:
messages_with_history = [
    {'role': 'system', 'content': 'You are a helpful assistant that answers programming questions in the style of a rancher from the western United States.' },
    {'role': 'user', 'content': 'Are semicolons optional in JavaScript?'},
    {'role': 'assistant', 'content': "Well, howdy there, partner! I reckon you're askin' about semicolons in JavaScript, eh? …, and may your code always be tidy!"},
    {'role': 'user', 'content': 'So to be clear, my code will not break with out them?'}
    ]

options = {
    'guardrails': True
}

from snowflake.cortex import Complete

response = Complete("llama3.1-405b", messages_with_history, options = options)

response


# Temperature

In [ ]:
options = {
    'guardrails': True,
    'temperature': 0.9
}

response = Complete("llama3.1-405b", messages_with_history, options = options)

response


# Top P

In [ ]:
options = {
    'guardrails': True,
    'top_p': 0.9
}

response = Complete("llama3.1-405b", messages_with_history, options = options)

response

# Max Tokens

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant that decides if a call transcript needs follow-up. You only respond by saying YES or NO.' },
    {'role': 'user', 'content': "Call Transcript: Your billing department is a disaster! I've been overcharged three times this month. Please refund me"}
]

options = {"max_tokens": 1}

response = Complete("llama3.2-1b", messages, options = options)

response

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'llama2-70b-chat',
    [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that decides if a call transcript needs follow-up. You only respond by saying YES or NO.'
        },
        {
            'role': 'user',
            'content': transcript
        }
    ],
    {'max_tokens': 1}
) AS followup_needed
FROM CALL_TRANSCRIPTS
LIMIT 10;
